## Numerator

In [ ]:
import numpy as np

Y = ['N', 'V', 'DT']
Y = ['S'] + Y # add in the start token

# Tokens

X = ['the', 'dog', 'barks']

# Vectorizers

vy = dict(zip(Y, range(len(Y))))
vx = dict(zip(X, range(len(X))))
vy_ = dict(zip(range(len(Y)), Y))
vx_ = dict(zip(range(len(X)), X))

def forward_backward_numerator(x, y, Λ):
    # Forward

    x_ = [None] + x
    y_ = ['S'] + y

    s = [1]
    z = [None]*len(x_)
    score = s[0]
    for i in range(1, 3):
        z[i] = np.exp(Λ[vx[x_[i]], vy[y_[i-1]], vy[y_[i]]])
        score *= z[i]
        s.append(score)

    score = s[-1]

    p = score
    
    # Backward
    
    dΛ = np.zeros_like(Λ, dtype=np.float64)
    dp = 1

    dscore = 1

    for i in range(2, 0, -1):
        dz = s[i-1] * dscore
        dΛ[vx[x_[i]], vy[y_[i-1]], vy[y_[i]]] = np.exp(Λ[vx[x_[i]], vy[y_[i-1]], vy[y_[i]]]) * dz
        dscore = z[i] * dscore
    
    return p, dΛ


Λ = np.random.randn(3, 4, 4)

forward_backward_numerator(['the', 'dog'], ['DT', 'N'], Λ)

# Denominator

In [ ]:
import numpy as np

Y = ['N', 'V', 'DT']
Y = ['S'] + Y # add in the start token

# Tokens

X = ['the', 'dog', 'barks']

# Vectorizers

vy = dict(zip(Y, range(len(Y))))
vx = dict(zip(X, range(len(X))))
vy_ = dict(zip(range(len(Y)), Y))
vx_ = dict(zip(range(len(X)), X))

def forward_backward_denominator(x, y, Λ):    
    # FORWARD

    x_ = [None] + x
    y_ = ['S'] + y

    Z = np.zeros([len(x_), 3, 3])
    A = np.zeros([len(x_), len(x_)])

    A[:, 1:2] = np.exp(Λ[vx[x_[1]], vy['S'], 1:].reshape(3, 1))

    for i in range(2, len(x_)):
        Z[i] = np.exp(Λ[vx[x_[i]], 1:, 1:].T)
        A[:, i:i+1] = Z[i] @ A[:, i-1:i]

    Z_denom = A[:, -1].sum()
    
    Z_inv = 1 / Z_denom
    
    p = Z_inv
    
    # BACKWARD
    
    dΛ = np.zeros_like(Λ, dtype=np.float64)
    dp = 1
    
    dpdZ_inv = dp
    
    dZ_inv = dpdZ_inv * dp
    
    dZ_invdZ = -(1 / Z_denom**2)
    
    dZ_denom = dZ_invdZ * dZ_inv

    da = np.full([3, 1], dZ_denom)
    
    for i in range(len(x_)-1, 1, -1):
        dZ = da @ A[:, i-1].reshape(1, 3)
        dΛ_T = np.exp(Λ[vx[x_[i]], 1:, 1:].T) * dZ
        dΛ[vx[x_[i]], 1:, 1:] = dΛ_T.T
        
        da = Z[i].T @ da

    dz1 = da.flatten()
    dΛ[vx[x_[1]], vy['S'], 1:] = np.exp(Λ[vx[x_[1]], vy['S'], 1:]) * dz1

    return p, dΛ

a1 = np.random.randn(3, 1)
Λ = np.random.randn(3, 4, 4)

forward_backward_denominator(['the', 'dog'], ['DT', 'N'], Λ=Λ)

# Full

In [ ]:
import numpy as np

Y = ['N', 'V', 'DT']
Y = ['S'] + Y # add in the start token

# Tokens

X = ['the', 'dog', 'barks']

# Vectorizers

vy = dict(zip(Y, range(len(Y))))
vx = dict(zip(X, range(len(X))))
vy_ = dict(zip(range(len(Y)), Y))
vx_ = dict(zip(range(len(X)), X))

def forward_backward(x, y, Λ):
    # Forward numerator

    x_ = [None] + x
    y_ = ['S'] + y

    s = [1]
    z = [None]*len(x_)
    score = s[0]
    for i in range(1, len(x_)):
        z[i] = np.exp(Λ[vx[x_[i]], vy[y_[i-1]], vy[y_[i]]])
        score *= z[i]
        s.append(score)

    score = s[-1]

    # Forward denominator

    Z = np.zeros([len(x_), 3, 3])
    A = np.zeros([3, len(x_)])

    A[:, 1:2] = np.exp(Λ[vx[x_[1]], vy['S'], 1:].reshape(3, 1))

    for i in range(2, len(x_)):
        Z[i] = np.exp(Λ[vx[x_[i]], 1:, 1:].T)
        A[:, i:i+1] = Z[i] @ A[:, i-1:i]

    Z_denom = A[:, -1].sum()
    
    Z_inv = 1 / Z_denom
    
    p = Z_inv

    p = score * Z_inv
    
    # Backward numerator
    
    dΛ = np.zeros_like(Λ, dtype=np.float64)
    dp = 1

    dscore = Z_inv

    for i in range(len(x_)-1, 0, -1):
        dz = s[i-1] * dscore
        dΛ[vx[x_[i]], vy[y_[i-1]], vy[y_[i]]] = np.exp(Λ[vx[x_[i]], vy[y_[i-1]], vy[y_[i]]]) * dz
        dscore = z[i] * dscore
    
    # Backward denominator
    
    dpdZ_inv = score
    
    dZ_inv = dpdZ_inv * dp
    
    dZ_invdZ = -(1 / Z_denom**2)
    
    dZ_denom = dZ_invdZ * dZ_inv

    da = np.full([3, 1], dZ_denom)
    
    for i in range(len(x_)-1, 1, -1):
        dZ = da @ A[:, i-1].reshape(1, 3)
        dΛ_T = np.exp(Λ[vx[x_[i]], 1:, 1:].T) * dZ
        dΛ[vx[x_[i]], 1:, 1:] += dΛ_T.T
        
        da = Z[i].T @ da

    dz1 = da.flatten()
    dΛ[vx[x_[1]], vy['S'], 1:] += np.exp(Λ[vx[x_[1]], vy['S'], 1:]) * dz1

    return p, dΛ


Λ = np.random.randn(3, 4, 4)

forward_backward(['the', 'dog', 'barks'], ['DT', 'N', 'V'], Λ=Λ)

# Gradient Check

In [ ]:
h = 0.00005

x = ['the', 'dog', 'barks']
y = ['DT', 'N', 'V']

Λ = np.random.randn(3, 4, 4)

for i in range(3):
    for j in range(4):
        for k in range(4):
            Λ[i, j, k] += h
            
            p, _ = forward_backward(x, y, Λ)
    
            Λ[i, j, k] -= 2*h
            
            p_, _ = forward_backward(x, y, Λ)
            
            Λ[i, j, k] += h
            
            numeric_gradient = (p - p_) / (2*h)
    
            _, dΛ = forward_backward(x, y, Λ)
    
            analytic_gradient = dΛ[i, j, k]
            
            error = abs(analytic_gradient - numeric_gradient)
            # print(error)
            
            assert error < 1e-4, error

'All clear!'

# Test

In [ ]:
import itertools

x = ['the', 'dog', 'barks']

Λ = np.random.randn(3, 4, 4)

sum = 0
for y_ in list(itertools.product(['N', 'V', 'DT'], repeat=3)):
    p, _ = forward_backward(x, list(y_), Λ)
    sum += p

sum

# Training

In [ ]:
import pandas as pd

learning_rate = 0.01

Λ = np.random.randn(3, 4, 4)

x = ['the', 'dog', 'barks']
y = ['DT', 'N', 'V']

probs = {y: [] for y in itertools.product(['N', 'V', 'DT'], repeat=3)}
for _ in range(100):
    for _ in range(100):
        _, dΛ = forward_backward(x, y, Λ)
        Λ += learning_rate*dΛ
    
    for y_ in list(itertools.product(['N', 'V', 'DT'], repeat=3)):
        p, _ = forward_backward(x, list(y_), Λ)
        probs[y_].append(p)

pd.DataFrame(probs).plot(xlabel='epoch', ylabel='$p(y|x)$', title='CRF Training');